In [1]:
import sys
print(sys.version)

3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]


In [2]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
from scipy import stats as ss
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score
from sklearn.neural_network import BernoulliRBM
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import neighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 



LEVERAGE = 1
from pandas_datareader import data as pdr

plt.rcParams['figure.figsize']  = (18, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4

In [3]:
def change_dataFrame(df,name):
    datas = []
    horas = []
    dias = []
    meses = []
    hora = []
    minuto = []
    
    for i in df[name]:
        i=i.split()
        datas.append(i[0])
        horas.append(i[1])
    for i in range(len(datas)):
        datas[i] = datas[i].split(".")
        dias.append(datas[i][0])
        meses.append(datas[i][1])
        horas[i] = horas[i].split(":")
        hora.append(horas[i][0])
        minuto.append(horas[i][1])
    df = df.drop(['Local time'],axis=1)
    df['Dia'] = pd.Series(dias,df.index)
    df['Mes'] = pd.Series(meses,df.index)
    df['Hora'] = pd.Series(hora,df.index)
    df['Minuto'] = pd.Series(minuto,df.index)
    
    return df
        

In [4]:
df = pd.read_csv('2019.csv')
df = change_dataFrame(df,'Local time')
#df=df[df['Mes']=="03"]
df = df.dropna()
df = df[['Open','High','Low','Close']]

In [5]:
df['Open-Close'] = df['Open']-df['Close']
df['High-Low'] = df['High']-df['Low']
df = df.dropna()
df = df[df['Open-Close'] != df['High-Low']]
X = df[['Open-Close','High-Low']]

In [6]:
Y = np.where(df['Close'].shift(-1) > df['Close'],1,-1)

In [7]:
split_percentage=0.7
split = int(split_percentage*len(df))
X_train = X[:split]
Y_train = Y[:split]

X_test = X[split:]
Y_test = Y[split:]

In [8]:
Qtde_Interacao2 = 20
kernelSVM = 'linear' #'rbf'
rs = 2
gammaSVM = 0.3
clf = SVC(gamma='scale')
clf.fit(X_train,Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [9]:
accuracy_train = accuracy_score(Y_train,clf.predict(X_train))
accuracy_test = accuracy_score(Y_test,clf.predict(X_test))

In [10]:
print(accuracy_train)

0.5439226960549433


In [11]:
print(accuracy_test)

0.5415729439793433


In [ ]:
df['Predicted_Signal'] = clf.predict(X)

#cumulative returns
LOG= ((df['Close'] / df['Close'].shift(1)-1)*LEVERAGE)+1
df['Real_Returns'] = np.log(LOG)
Cumulative_returns = df[split:]['Real_Returns'].cumsum()*100

#retornos acumulados da estratégia

df['Strategy_Returns'] = df['Real_Returns'] * df['Predicted_Signal'].shift(1)
Cum_Strategy_Returns = df[split:]['Strategy_Returns'].cumsum()*100

#Plotando os resultados 


plt.figure(figsize=(20,10))
plt.plot(Cumulative_returns,color='r',label='Real Returns')
plt.plot(Cum_Strategy_Returns,color='g',label='Strategy Returns')

plt.xlabel('tempo (m)')
plt.ylabel('ganho (%)')

plt.legend()
plt.show()

In [13]:
#sharpe ratio acumulado

Std = Cum_Strategy_Returns.std()
Sharpe = (Cum_Strategy_Returns-Cumulative_returns)/Std
Sharpe = Sharpe.mean()
print('Sharpe ratio: %.2f'%Sharpe)

Sharpe ratio: 2.52


In [14]:
print(np.exp(df[split:]['Strategy_Returns'].sum()))

1.0643560064991608
